In [65]:
import tensorflow as tf
import datetime, os
from model import *
from data import *

#### Unpack videos and labels

In [66]:
unpack_video('data/larynx/train', video_folder='video', image_folder='image', target_size=(256, 256))
unpack_tif('data/larynx/train', tif_folder='tifs', label_folder='label', target_size=(256, 256))

#### Initialize the model (U-Net)

In [67]:
model = unet()
model_checkpoint = ModelCheckpoint('unet_larynx.hdf5', monitor='loss', verbose=1, save_best_only=True)

#### Prepare training dataset

In [68]:
training_dataset = training_dataset_generator(2,
                                              'data/larynx/train',
                                              'image',
                                              'label',
                                              dict([]),
                                              save_to_dir=None)

In [69]:
# Set up TensorBoard callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    verbose=1,
    mode='min',
    restore_best_weights=True
)


# Train the model using the training dataset and TensorBoard callback
history = model.fit(training_dataset, 
          steps_per_epoch=10, 
          epochs=100, 
          callbacks=[model_checkpoint, tensorboard_callback, early_stopping])

Found 55 images belonging to 1 classes.
Found 55 images belonging to 1 classes.
Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 24.4827 - accuracy: 0.9632
Epoch 1: loss improved from inf to 24.48268, saving model to unet_larynx.hdf5
10/10 [==============================] - 21s 2s/step - loss: 24.4827 - accuracy: 0.9632
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 0.2026 - accuracy: 0.9642
Epoch 2: loss improved from 24.48268 to 0.20256, saving model to unet_larynx.hdf5
10/10 [==============================] - 20s 2s/step - loss: 0.2026 - accuracy: 0.9642
Epoch 3/100
10/10 [==============================] - ETA: 0s - loss: 0.1399 - accuracy: 0.9821
Epoch 3: loss improved from 0.20256 to 0.13987, saving model to unet_larynx.hdf5
10/10 [==============================] - 19s 2s/step - loss: 0.1399 - accuracy: 0.9821
Epoch 4/100
10/10 [==============================] - ETA: 0s - loss: 0.1325 - accuracy: 0.9759
Epoch 4: loss improved from 0.13987 

#### Testing the model

In [44]:
# testing the model
test_dataset = test_dataset_generator("data/larynx/test", num_image=5)
results = model.predict(test_dataset, 3, verbose=1)
saveResult("data/larynx/test", results)


5/5 [==============================] - 1s 172ms/step


c:\Users\perseusf\anaconda3\lib\site-packages\keras\src\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


55/55 [==============================] - 9s 172ms/step


c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\0_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\1_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\2_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\3_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/segment/segment_tmp\4_predict.png is a low contrast image
  io.imsave(os.path.join(save_path, "%d_predict.png" % i), img)
c:\dev\git\larynx_NN_model\data.py:308: UserWarning: data/larynx/